In [ ]:
# Install dependencies
#!pip install -q torch transformers huggingface_hub accelerate llama-cpp-python[cuda]

# Download Mistral 7B (4-bit quantized GGUF)
#!wget -q https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf -O mistral-7b.Q4_K_M.gguf

# Load model and run inference
from llama_cpp import Llama

# Initialize the model
llm = Llama(model_path="mistral-7b-instruct-v0.1.Q4_K_M.gguf", n_gpu_layers=35, n_ctx=4096, verbose=False)

# Run a simple German QA example
question = "Was ist die Hauptstadt der Schweiz?"
response = llm(f"Antworte auf Deutsch: {question}")

# Print response
print(response["choices"][0]["text"])


llama_init_from_model: n_ctx_per_seq (4096) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


 Bern ist die Hauptstadt der Schweiz.


In [10]:
# Load model and run inference
from llama_cpp import Llama

# Initialize the model
llm = Llama(model_path="mistral-7b-instruct-v0.1.Q4_K_M.gguf", n_gpu_layers=35, n_ctx=32768, verbose=False)

# Run a simple German QA example
question = "Was ist die Hauptstadt der Schweiz?"
response = llm(f"Antworte auf Deutsch: {question}")

# Print response
print(response["choices"][0]["text"])

# System prompt with instructions for the workflow assistant
system_prompt = """
You are a helpful assistant for setting up the CARDIOHELP-i medical device. 
Follow these rules:
1. Detect the user's language and respond in the same language
2. Guide the user through the setup process step by step
3. Only provide the next step after the user confirms with "ok" or equivalent in their language
4. Be concise and clear with instructions
5. Maintain a professional, supportive tone

The setup workflow has the following steps:
1. Mount the holder and any necessary accessories
2. Position and connect the CARDIOHELP-i
3. Connect the sensors
4. Turn on the CARDIOHELP-i and allow it to run a self-test

Wait for user confirmation after each step before proceeding to the next one.
"""

current_step = 0
user_input   = "ok"



prompt = f"""[INST] {system_prompt}

Current status: Step {current_step+1} of 5
User input: {user_input} [/INST]
"""


# Create the prompt for the model
#prompt = f"""<s>[INST] {system_prompt}

#Current status: Step {current_step+1} of 5
#User input: {user_input} [/INST]
#"""
#Get model response

response = llm(prompt, max_tokens=512, temperature=0.1)
model_response = response["choices"][0]["text"].strip()

print(response)

 Bern ist die Hauptstadt der Schweiz.
{'id': 'cmpl-3c27f45c-4fcf-48a4-a88a-d6bb4d41afa6', 'object': 'text_completion', 'created': 1743093632, 'model': 'mistral-7b-instruct-v0.1.Q4_K_M.gguf', 'choices': [{'text': "Great! Let's move on to step 2. Please position and connect the CARDIOHELP-i to the holder and any necessary accessories.", 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 205, 'completion_tokens': 32, 'total_tokens': 237}}


In [3]:
# Load model and run inference
from llama_cpp import Llama

# Initialize the model
llm = Llama(model_path="mistral-7b-instruct-v0.1.Q4_K_M.gguf", n_gpu_layers=35, n_ctx=32768, verbose=False)
#llm = Llama(model_path="gemma-2-2b-it-Q4_K_M.gguf", verbose=False)

# Define the workflow with dynamic step-by-step guidance
workflow_steps = [
    
    "Mount the holder and any necessary accessories",
    "Position and connect the CARDIOHELP",
    "Connect the sensors",
    "Turn on the CARDIOHELP and allow it to run a self-test"
]

current_step = 0
user_language = "english"  # You might want to update this dynamically with user input

while current_step < len(workflow_steps):

    step_instruction = workflow_steps[current_step]
    
    # System prompt with instructions for the workflow assistant
    system_prompt = f"""
    You are a helpful assistant for setting up the CARDIOHELP medical device.
    Follow these rules:
    1. Detect the user's language and respond in the same language.
    2. Guide the user through the setup process step by step
    3. Only provide the next step after the user confirms with "ok".
    4. Be concise and clear with instructions
    5. Maintain a professional, supportive tone and friendly.

    Current status: Step {current_step + 1} of 5
    User language: {user_language}
    only introduce yourself if you are in currect status 1
    """

    # Prompt to provide the current step's instruction
    prompt = f"""[INST] {system_prompt} Instruction for step {current_step + 1}: {step_instruction} [/INST]"""

    # Get model response
    response = llm(prompt, max_tokens=512, temperature=0.1)
    model_response = response["choices"][0]["text"].strip()

    # Print the response for the current step
    print(f"Step {current_step + 1}: {model_response}")

    # Ask for user confirmation before proceeding

    # call to funtion to read users voice
    #user_input = read_users_voice()    # this funtion delivers a charachterschain
    # user_input = input("Please confirm (type 'ok' to continue): ").strip()

    user_input = 'ok'
    
    print('-- user: ok')

    if user_input.lower() in ["ok", "yes", "ja", "oui"]:  # Adjust for multiple languages
        current_step += 1
    else:   
        print("Please confirm to proceed.")



Step 1: Hello! I'm here to assist you with setting up the CARDIOHELP medical device. Let's start with step 1. Please mount the holder and any necessary accessories. Once you've completed this step, please let me know and we can move on to the next step.
-- user: ok
Step 2: Hello! I'm here to help you set up the CARDIOHELP medical device. Let's start with step 2. Please position the device on a flat surface and connect it to the power source. Make sure the device is securely plugged in and that the power cord is not damaged. Once you have connected the device, please confirm with "ok" or equivalent in your language.
-- user: ok
Step 3: Hello! I'm here to assist you with setting up the CARDIOHELP medical device. We're currently on step 3 of 5. Let's move on to connecting the sensors.

To connect the sensors, please follow these steps:

1. Locate the sensor pads on the device. They should be labeled "front" and "back."
2. Place the sensor pads on the appropriate locations on your body. Th

In [1]:
# Load model and run inference
from llama_cpp import Llama

# Initialize the model
llm = Llama(model_path="mistral-7b-instruct-v0.1.Q4_K_M.gguf", n_gpu_layers=35, n_ctx=32768, verbose=False)

# Define the workflow steps
workflow_steps = [
    "Mount the holder and any necessary accessories",
    "Position and connect the CARDIOHELP",
    "Connect the sensors",
    "Turn on the CARDIOHELP and allow it to run a self-test"
]

current_step = 0
user_language = "English"  # This could be dynamically detected

def interpret_user_response(user_input):
    """Ask the model if the user confirmed the step and enforce a strict 'yes' or 'no' answer."""
    confirmation_prompt = f"""
    You are an intelligent assistant guiding a user through setting up the CARDIOHELP medical device.
    The user just responded: "{user_input}"
    
    Does this mean they have completed the step?
    Answer ONLY "yes" or "no". Do NOT include any other text.
    """

    response = llm(f"[INST] {confirmation_prompt} [/INST]", max_tokens=2, temperature=0.1)
    model_reply = response["choices"][0]["text"].strip().lower()

    print(f"DEBUG: Model interpreted user response as -> '{model_reply}'")  # Debugging

    if model_reply.startswith("yes"):
        return "yes"
    elif model_reply.startswith("no"):
        return "no"
    else:
        return "unclear"  # Handle unexpected responses

while current_step < len(workflow_steps):

    step_instruction = workflow_steps[current_step]
    
    # System prompt with instructions
    system_prompt = f"""
    You are a helpful assistant for setting up the CARDIOHELP medical device.
    Follow these rules:
    1. Detect the user's language and respond in the same language
    2. Guide the user through the setup process step by step
    3. Only provide the next step after the user confirms
    4. Be concise and clear with instructions
    5. Maintain a professional, supportive, and friendly tone

    Current status: Step {current_step + 1} of {len(workflow_steps)}
    User language: {user_language}
    """

    # Generate model response for the step instruction
    prompt = f"""[INST] {system_prompt} Instruction for step {current_step + 1}: {step_instruction} [/INST]"""
    response = llm(prompt, max_tokens=512, temperature=0.1)
    model_response = response["choices"][0]["text"].strip()

    # Print the response for the current step
    print(f"Step {current_step + 1}: {model_response}")

    # Simulated voice input function
    # user_input = read_users_voice()
    #user_input = input("Your response: ").strip()  # Replace with voice processing in the future
    user_input = "finished"

    print(f"-- user: {user_input}")

    # Let the model decide if we move to the next step
    model_decision = interpret_user_response(user_input)
    
    if model_decision == "yes":
        current_step += 1
    elif model_decision == "no":
        print("Please complete this step before continuing.")
    else:
        print("I'm not sure if you confirmed. Please answer clearly.")

print("Setup complete! ✅")


Step 1: Hello! I'm here to help you set up your CARDIOHELP medical device. Let's start with step 1. Please mount the holder and any necessary accessories. Make sure to follow the manufacturer's instructions carefully. Once you've completed this step, please let me know and we can move on to the next step.
-- user: finish
DEBUG: Model interpreted user response as -> 'no.'
Please complete this step before continuing.


KeyboardInterrupt: 

In [ ]:
# Load model and run inference
import random
from llama_cpp import Llama
import re

# Initialize the model
llm = Llama(model_path="mistral-7b-instruct-v0.1.Q4_K_M.gguf", n_gpu_layers=35, n_ctx=32768, verbose=False)
#llm = Llama(model_path="em_german_7b_v01.Q8_0.gguf", n_ctx= 4096,verbose=False )
#llm = Llama(model_path="em_german_7b_v01.Q8_0.gguf", n_ctx=4096, verbose=True)

# Define the workflow steps
workflow_steps = [
    "Mount the holder and any necessary accessories.",
    "Position and connect the CARDIOHELP.",
    "Connect the sensors.",
    "Turn on the CARDIOHELP and allow it to run a self-test."
]

current_step = 0
user_language = "English"  # This could be dynamically detected

def interpret_user_response(user_input):
    """Ask the model if the user confirmed the step and enforce a strict 'yes' or 'no' answer."""
    confirmation_prompt = f"""
    You are an intelligent assistant helping the user set up the CARDIOHELP medical device.
    
    The user responded: "{user_input}"
    
    Decide whether this means they have finished the step. Examples of confirmations:
    - "Done!"
    - "Yes, I finished."
    - "or anything else that means finished"
    
    Answer ONLY with "yes" or "no". Do NOT include any other text.
    """

    response = llm(f"[INST] {confirmation_prompt} [/INST]", max_tokens=2, temperature=0.1)
    model_reply = response["choices"][0]["text"].strip().lower()

    # Debugging output
    #print(f"DEBUG: Raw model response -> '{model_reply}'")  

    # Regex pattern to check if the response starts with 'yes' or 'no'
    if re.match(r"^\s*yes\b", model_reply):
        return "yes"
    elif re.match(r"^\s*no\b", model_reply):
        return "no"
    else:
        return "unclear"  # Handle unexpected responses

while current_step < len(workflow_steps):

    step_instruction = workflow_steps[current_step]

    #if current_step == 0:
    
        # System prompt with instructions
    system_prompt = f"""
        You are a helpful assistant for setting up the CARDIOHELP medical device.
        just say hello if you are in the current step number 1.
        for the rest just follow the workflow
        Follow these rules:
        1. Detect the user's language and respond in the same language.
        2. Guide the user through the setup process step by step.
        3. Only provide the next step after the user confirms.
        4. Be concise and clear with instructions.
        5. Maintain a professional, supportive, and friendly tone.

        Current status: Step {current_step + 1} of {len(workflow_steps)}
        User language: {user_language}
        lastly: Detect the user's language and respond in the same language.
        """
    #else:
        
    #system_prompt = f"""Current status: Step {current_step + 1} of {len(workflow_steps)}
    #User language: {user_language}
    #lastly: Detect the user's language and respond in the same language.
    #"""

    # Generate model response for the step instruction
    prompt = f"""[INST] {system_prompt} Instruction for step {current_step + 1}: {step_instruction} [/INST]"""
    response = llm(prompt, max_tokens=512, temperature=0.1)
    model_response = response["choices"][0]["text"].strip()

    # Print the response for the current step
    print(f"\nStep {current_step + 1}: {model_response}")

    # Simulated voice input function
    #user_input = input("Your response: ").strip()  # Replace with voice processing in the future

    #user_input = 'finished'
    #user_input = random.choice(["finished","done","completed", "ok"])
    user_input = random.choice(["fertig", "erledigt", "abgeschlossen", "ok"])

    print(f"-- user: {user_input}")

    # Let the model decide if we move to the next step
    model_decision = interpret_user_response(user_input)
    
    if model_decision == "yes":
        current_step += 1
    elif model_decision == "no":
        print("Please complete this step before continuing.")
    else:
        print("I'm not sure if you confirmed. Please answer clearly (e.g., 'Done', 'Finished', 'Weiter').")

print("Setup complete!")


llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from em_german_7b_v01.Q8_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32              = 3


Step 1: 
-- user: abgeschlossen


llama_perf_context_print:        load time =   10858.49 ms
llama_perf_context_print: prompt eval time =    5641.00 ms /   108 tokens (   52.23 ms per token,    19.15 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    5643.21 ms /   109 tokens
Llama.generate: 6 prefix-match hit, remaining 186 prompt tokens to eval
llama_perf_context_print:        load time =   10858.49 ms
llama_perf_context_print: prompt eval time =    9673.03 ms /   186 tokens (   52.01 ms per token,    19.23 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    9674.15 ms /   187 tokens
Llama.generate: 6 prefix-match hit, remaining 107 prompt tokens to eval



Step 1: 
-- user: fertig


llama_perf_context_print:        load time =   10858.49 ms
llama_perf_context_print: prompt eval time =    6414.49 ms /   107 tokens (   59.95 ms per token,    16.68 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    6415.31 ms /   108 tokens
Llama.generate: 6 prefix-match hit, remaining 186 prompt tokens to eval
llama_perf_context_print:        load time =   10858.49 ms
llama_perf_context_print: prompt eval time =   12647.81 ms /   186 tokens (   68.00 ms per token,    14.71 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   12648.90 ms /   187 tokens
Llama.generate: 6 prefix-match hit, remaining 107 prompt tokens to eval



Step 1: 
-- user: fertig


llama_perf_context_print:        load time =   10858.49 ms
llama_perf_context_print: prompt eval time =   29646.00 ms /   107 tokens (  277.07 ms per token,     3.61 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   29647.10 ms /   108 tokens
Llama.generate: 6 prefix-match hit, remaining 186 prompt tokens to eval


# german (em_mistral)

In [6]:
# Load model and run inference
import random
from llama_cpp import Llama
import re

# Initialize the model with more appropriate settings
llm = Llama(
    model_path="em_german_7b_v01.Q8_0.gguf", 
    n_ctx=4096, 
    verbose=False,
    n_batch=512,  # Increase batch size for better performance
    echo=False,   # Don't include the prompt in the response
    temperature=0.5  # Add some randomness to help generate responses
)

# Define the workflow steps - in German since we're using a German model
workflow_steps = [
    "Montieren Sie die Halterung und das notwendige Zubehör.",
    "Positionieren und schließen Sie das CARDIOHELP-Gerät an.",
    "Verbinden Sie die Sensoren.",
    "Schalten Sie das CARDIOHELP-Gerät ein und lassen Sie es einen Selbsttest durchführen."
]

current_step = 0
user_language = "German"

def interpret_user_response(user_input):
    """Simplified function that doesn't rely on model interpretation"""
    # Simple pattern matching instead of using the model
    user_input = user_input.lower()
    
    # List of German words indicating completion
    confirmation_words = [
        "fertig", "erledigt", "abgeschlossen", "ok", "ja",
        "gemacht", "geschafft", "bereit", "weiter"]
    
    # Check if any confirmation word is in the user input
    for word in confirmation_words:
        if word in user_input:
            #print(f"Confirmation detected: '{word}' in '{user_input}'")
            return "yes"
    
    return "no"

while current_step < len(workflow_steps):
    step_instruction = workflow_steps[current_step]
    
    # Simplified German system prompt
    system_prompt = f"""
    Du bist ein hilfreicher Assistent für die Einrichtung des CARDIOHELP-Medizingeräts.
    
    Aktueller Status: Schritt {current_step + 1} von {len(workflow_steps)}
    
    Sei präzise und klar in deinen Anweisungen.
    """

    # Custom prompt formatting based on the model type
    # Try a simple format first
    full_prompt = f"<s>Bitte gib Anweisungen für Schritt {current_step + 1}: {step_instruction}</s>"
    
    try:
        # Generate a response
        response = llm(
            full_prompt,
            max_tokens=256,
            temperature=0.5,  # Higher temperature for more creative responses
            stop=["</s>", "<s>"]  # Common stop tokens
        )

        response = response['choices'][0]['text'].strip()
        
        # Print whatever we get back, regardless of format
        #print(f"\nSchritt {current_step + 1}:")
        print(response if isinstance(response, str) else str(response))
        
        # For testing purposes, always provide a confirmation
        user_input = random.choice(["fertig", "erledigt", "abgeschlossen", "ok"])
        print(f"-- Benutzer: {user_input}")
        
        # Using our simpler interpretation function
        model_decision = interpret_user_response(user_input)
        #print(f"Entscheidung: {model_decision}")
        
        # Always move to the next step in test mode
        current_step += 1
        #print("Nächster Schritt...")
            
    except Exception as e:
        print(f"Error generating response: {e}")
        # Increment step to avoid infinite loop in case of error
        current_step += 1

print("Einrichtung abgeschlossen!")

Bitte gib Anweisungen für Schritt 2: Montiere den Motor und die Steuerung.
-- Benutzer: ok
Schritt 2: Positionieren und schließen Sie das CARDIOHELP-Gerät an

Um das CARDIOHELP-Gerät an Ihrem Gerät zu positionieren und zu schließen, befolge diese Anweisungen:

1. Stelle sicher, dass das CARDIOHELP-Gerät aus dem Aufbewahrungsbehälter entnommen und auf Ihrem Gerät liegt.
2. Das CARDIOHELP-Gerät sollte in der horizontalen Position liegen.
3. Die Klemmen zur Verbindung des Geräts zum Gerät Ihres Patienten sind oben und unten sichtbar.
4. Schließen Sie das CARDIOHELP-Gerät an die Klemmen am Gerät Ihres Patienten.
5. Das CARDIOHELP-Gerät sollte sicher an Ihrem Gerät angekoppelt sein.

Nun ist das CARDIOHELP-Gerät an Ihrem Gerät angekoppelt und bereit, die erforder
-- Benutzer: erledigt
Wenn du den Anweisungen in der vorherigen Erklärung folgst, solltest du die Sensoren wie in den Bildern unten gezeigt verbunden haben.
-- Benutzer: ok
Wenn Sie das CARDIOHELP-Gerät erhalten haben, folgen Sie b

In [ ]:
# example how to run the tts 

# # From your parent directory script:
# from tts import generate_speech

# # Generate speech
# output = generate_speech(
#     text="Überprüfen Sie ob alle in der Lieferungen enthaltenen Komponenten vorhanden sind.",
#     output_file="output.wav",
#     reference_audio="Germany_second.wav",  # Note: path is relative to where you run the script
#     language="de"
# )

# print(f"Speech generated and saved to {output}")

: 